<a href="https://colab.research.google.com/github/filipediniz/Coursera_Capstone/blob/master/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The problem to be solved:**

In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it?

The city is Irvine, California.

**1. A description of the problem and a discussion of the background. (15 marks)**

The scenario is that a immigrant is coming to South California and he wants to open any kind of restaurant. He is moving specifically to a town called Irvine, which is located in Orange County. Several companies are there, over 60 malls and over 40 villages are also part of the picture. But where should he open this restaurant considering all business that are located near those villages. The main audience to this storytelling line is business persons who recently moved into town.

**2. A description of the data and how it will be used to solve the problem. (15 marks)**

The data was retrieved using Foursquare API and it describes what kind of business that are located within 1 mile from each village. The goal here is to make an analysis from the distribution of venues from all over the city and specific to each village.

# Introduction

In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it?

The city is Irvine, California.

The scenario is that a immigrant is coming to South California and he wants to open any kind of restaurant. He is moving specifically to a town called Irvine, which is located in Orange County. Several companies are there, over 60 malls and over 40 villages are also part of the picture. But where should he open this restaurant considering all business that are located near those villages. The main audience to this storytelling line is business persons who recently moved into town.

The data was retrieved using Foursquare API and it describes what kind of business that are located within 1 mile from each village. The goal here is to make an analysis from the distribution of venues from all over the city and specific to each village.

The first thing to do is to make sure all necessary packages are properly installed.

In [1]:
!pip install requests_html
!pip install pixiedust
!pip install geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

/bin/bash: conda: command not found


And then it is necessary to import all packages

In [2]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import pixiedust
import requests
from requests_html import HTMLSession
import re as regex

import datetime as dt
from pandas_datareader import data

from IPython.display import clear_output

import pickle

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Pixiedust database opened successfully


# Data description

Next we use the Notebook to build the code to scrape the Irvine Wikipedia page, https://en.wikipedia.org/wiki/Irvine,_California, in order to obtain the data that is input for a table of Irvine villages and to transform the data into a pandas dataframe.

The main data consists of a dataframe containing all Irvine villages. Besides that, one also needs to retrieve their coordinates (longitude and latitude), not to mention all venues around them along with their type.

In [0]:
from bs4 import BeautifulSoup

session = HTMLSession()
DF = pd.DataFrame()
ret = session.get('https://en.wikipedia.org/wiki/Irvine,_California')    
try:    
    soup = BeautifulSoup(ret.text, 'html.parser')
except:
    print('Página sem dados')

In [4]:
html_objs = soup.find_all("h4")
villages = ''
for html_obj in html_objs:
  print(html_obj.text)
  if html_obj.text == 'Villages':    
    villages = html_obj.next_element.next_element.next_element.next_element.next_element.next_element.next_element
    break
neighborhoods = []
for village in list((list(villages.children)[1]).children):
  if village.name != None:    
    neighborhoods.append(village.text.split(' (')[0].split('[')[0])    

Villages


In [5]:
DF = pd.DataFrame()
DF['Neighborhood'] = neighborhoods
DF

,Neighborhood
0,El Camino Glen
1,College Park
2,The Colony
3,Columbus Grove
4,Cypress Village
5,Deerfield
6,East Irvine
7,El Camino Real
8,Greentree
9,Irvine Groves


Then it is necessary to retrieve coordinates (latitude and longitude) from each village. In order to do so, the Nominatim package was used.

In [6]:
#@title
import geocoder # import geocoder
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except: #GeocoderTimedOut:
        print('Error getting coordinates.')
        return None        

DF['Latitude'] =  None
DF['Longitude'] = None

for r in range(0, DF.shape[0]):
  print('Current row: {}'.format(r))
  # initialize your variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  g = None
  print('Attempting to retrieve coordinates for address {}...'.format('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood'])))
  geolocator = Nominatim(user_agent="irvine_explorer")
  # g = geolocator.geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )          
  g = do_geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )            
  if (type(g) != type(None)):    
    print('Latitude = {}; longitude = {}'.format(g.latitude, g.longitude))  
    DF.at[r, 'Latitude'] = g.latitude
    DF.at[r, 'Longitude'] = g.longitude    

Current row: 0
Attempting to retrieve coordinates for address El Camino Glen, Irvine, California...
Latitude = 33.6960782; longitude = -117.790868
Current row: 1
Attempting to retrieve coordinates for address College Park, Irvine, California...
Latitude = 33.7049508; longitude = -117.79327516984964
Current row: 2
Attempting to retrieve coordinates for address The Colony, Irvine, California...
Latitude = 33.75507555; longitude = -117.80374469402437
Current row: 3
Attempting to retrieve coordinates for address Columbus Grove, Irvine, California...
Latitude = 33.7045487; longitude = -117.7637001
Current row: 4
Attempting to retrieve coordinates for address Cypress Village, Irvine, California...
Latitude = 33.6866608; longitude = -117.7579676
Current row: 5
Attempting to retrieve coordinates for address Deerfield, Irvine, California...
Latitude = 33.6929179; longitude = -117.7907331
Current row: 6
Attempting to retrieve coordinates for address East Irvine, Irvine, California...
Latitude = 

In [7]:
DF

,Neighborhood,Latitude,Longitude
0,El Camino Glen,33.6961,-117.791
1,College Park,33.705,-117.793
2,The Colony,33.7551,-117.804
3,Columbus Grove,33.7045,-117.764
4,Cypress Village,33.6867,-117.758
5,Deerfield,33.6929,-117.791
6,East Irvine,33.6639,-117.743
7,El Camino Real,33.6942,-117.785
8,Greentree,33.6976,-117.79
9,Irvine Groves,33.7055,-117.783


However the Nominatim package is not so reliable so it is necessary to drop all rows from which contains NaN values.

In [8]:
DF.dropna(inplace=True)
DF.reset_index()

,index,Neighborhood,Latitude,Longitude
0,0,El Camino Glen,33.6961,-117.791
1,1,College Park,33.705,-117.793
2,2,The Colony,33.7551,-117.804
3,3,Columbus Grove,33.7045,-117.764
4,4,Cypress Village,33.6867,-117.758
5,5,Deerfield,33.6929,-117.791
6,6,East Irvine,33.6639,-117.743
7,7,El Camino Real,33.6942,-117.785
8,8,Greentree,33.6976,-117.79
9,9,Irvine Groves,33.7055,-117.783


In [9]:
DF

,Neighborhood,Latitude,Longitude
0,El Camino Glen,33.6961,-117.791
1,College Park,33.705,-117.793
2,The Colony,33.7551,-117.804
3,Columbus Grove,33.7045,-117.764
4,Cypress Village,33.6867,-117.758
5,Deerfield,33.6929,-117.791
6,East Irvine,33.6639,-117.743
7,El Camino Real,33.6942,-117.785
8,Greentree,33.6976,-117.79
9,Irvine Groves,33.7055,-117.783


In order to visualize irvine area within a interactive map, it requires the latitude and longitude coordinates. They were retrieved using the Nominatim package.

In [10]:
address = 'Irvine'

geolocator = Nominatim(user_agent="irvine_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Irvine are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Irvine are 33.6856969, -117.8259819.


Then a map in created using the Folium package.

In [11]:
# create map of irvine using latitude and longitude values
map_irvine = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(DF['Latitude'], DF['Longitude'], DF['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_irvine)  
    
map_irvine

In order to use the Foursquare API, it is necessary to input client ID, client secret, version and limit.

In [12]:
CLIENT_ID = '4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI' # your Foursquare ID
CLIENT_SECRET = 'UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI
CLIENT_SECRET:UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44


This function aims at listing interesting places that are located within 1600 meters (i.e. 1 mile) radius from the point defined by some pair of latitude / longitude values.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=1600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

This list is the result from using the function above.

In [14]:
irvine_venues = getNearbyVenues(names=DF['Neighborhood'],
                                   latitudes=DF['Latitude'],
                                   longitudes=DF['Longitude']
                                  )

El Camino Glen
College Park
The Colony
Columbus Grove
Cypress Village
Deerfield
East Irvine
El Camino Real
Greentree
Irvine Groves
Harvard Square
Laguna Altura
Lambert Ranch
Northpark
Northpark Square
Northwood
Oak Creek
Old Towne Irvine
Orangetree
Orchard Hills
Park Lane
Parkcrest
Parkside
Pavilion Park
Portola Springs
Quail Hill
Racquet Club
The Ranch
Rancho San Joaquin
Rosegate
Stonegate
Shady Canyon
Turtle Ridge
Turtle Rock
University Hills
University Park
University Town Center
Walnut
West Irvine
Westpark
The Willows
Windwood
Woodbridge
Woodbury
Woodbury East


The resulting dataframe, which, besides the Neighborhood name, also contains latitude, longitude from both the neighborhoods themselves and the venues, including their category.

In [15]:
#irvine_venues.set_index(list(irvine_venues)[0], inplace=True)
irvine_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,El Camino Glen,33.696078,-117.790868,Lovely Nails,33.696466,-117.798373,Spa
1,El Camino Glen,33.696078,-117.790868,All That Barbecue,33.696866,-117.798111,Korean Restaurant
2,El Camino Glen,33.696078,-117.790868,Trader Joe's,33.706082,-117.788254,Grocery Store
3,El Camino Glen,33.696078,-117.790868,The Habit Burger Grill,33.704707,-117.786916,Burger Joint
4,El Camino Glen,33.696078,-117.790868,Sam Woo Seafood Restaurant,33.695909,-117.798085,Chinese Restaurant


The next step would be to format the above dataframe using the one hot encoding.

In [16]:
# one hot encoding
irvine_onehot = pd.get_dummies(irvine_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
irvine_onehot['Neighborhood'] = irvine_venues['Neighborhood'] 

# move neighborhood column to the first column
irvine_onehot = pd.concat([irvine_onehot['Neighborhood'].to_frame(), irvine_onehot.drop(columns=['Neighborhood'], axis=1)], axis=1)
irvine_onehot.head()

,Neighborhood,ATM,Accessories Store,Airport,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Cheese Shop,...,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


The problem is that this format is very sparse making the dimension of the dataframe to be very large.

In [17]:
irvine_onehot.shape

(2880, 209)

So one needs to count all types of venues within the city grouped by village area.

In [18]:
irvine_grouped = irvine_onehot.groupby('Neighborhood').sum().reset_index()
irvine_grouped

,Neighborhood,ATM,Accessories Store,Airport,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Cheese Shop,...,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,College Park,0,0,0,0,0,0,0,0,0,0,0,1,1,6,4,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,2,0,0,1,0,0,1,0,...,0,3,0,0,0,0,0,0,1,0,0,1,3,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
1,Columbus Grove,1,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1
2,Cypress Village,0,0,0,0,0,0,0,0,1,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,1,0,0,1,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,2,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1
3,Deerfield,0,0,0,0,0,0,0,0,0,0,0,0,1,3,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,2,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0
4,East Irvine,0,1,0,3,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,2,0,1,0,0,...,0,2,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,1,0,0,0,0,0,0
5,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,1,6,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,2,0,0,1,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0
6,El Camino Real,0,0,0,0,0,0,0,0,0,0,0,0,1,8,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,2,0,0,1,0,0,0,0,...,0,3,0,0,0,0,0,0,1,0,0,0,4,0,0,1,0,2,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1
7,Greentree,0,0,0,0,0,0,0,0,0,0,0,0,1,6,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,2,0,0,1,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0
8,Harvard Square,0,0,0,1,0,0,0,0,1,0,0,0,1,4,2,0,0,0,0,1,1,0,0,1,0,0,1,2,1,1,0,4,0,0,0,0,1,0,0,...,0,3,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,1,1,0,1
9,Irvine Groves,0,0,0,0,0,0,0,0,0,0,0,1,1,4,3,0,2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,2,0,0,1,0,0,0,0,...,0,2,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0


# Methodology

At this point, all necessary data to solve this problem is already gathered. The method to get to an answer is:



1.   List the number of venues per type.
2.   Sort the list to get the most frequent type of venues.
3.   Get the most frequent type of restaurant. That would be our goal type of restaurant.
4.   List the villages around which there is no Chinese restaurant.
5.   List the top villages with the largest number of venues around them.
6.   For each village from step 5, check if it is contained in step 4 list. 
7.   If the check from step 6 is true, then this village has a lot of venues, which means that there are several points of interest and it is not a "desert", but is lacks a Chinese restaurant.







And the total number of each kind of venue.

In [19]:
irvine_total = irvine_grouped.sum().reset_index()
irvine_total.columns = ['Type', 'Total']
irvine_total.drop(axis=0, index = 0, inplace=True)
irvine_total

,Type,Total
1,ATM,8
2,Accessories Store,3
3,Airport,1
4,American Restaurant,31
5,Amphitheater,2
...,...,...
204,Wine Bar,6
205,Wine Shop,2
206,Wings Joint,9
207,Women's Store,2


In [20]:
list(irvine_total['Type'])

['ATM',
 'Accessories Store',
 'Airport',
 'American Restaurant',
 'Amphitheater',
 'Arcade',
 'Art Gallery',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Automotive Shop',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Basketball Court',
 'Beach',
 'Big Box Store',
 'Bike Shop',
 'Bistro',
 'Board Shop',
 'Bookstore',
 'Boutique',
 'Bowling Alley',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Business Service',
 'Cafeteria',
 'Café',
 'Cajun / Creole Restaurant',
 'Candy Store',
 'Cantonese Restaurant',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chiropractor',
 'Church',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Quad',
 'Comedy Club',
 'Concert Hall',
 'Construction & Landscaping',
 'Convenience Store',
 'Cosmetics Shop',
 'Cuban Restaurant',
 'Dance Studio',
 'Deli / Bodega',
 'Department Store',
 'Dessert Shop',
 'Dim Sum Restaurant',
 'Diner',
 'Dog

And sort all kinds of venues by its quantity.

In [21]:
irvine_total.sort_values(by='Total',  ascending=False, inplace=True)
irvine_total.reset_index(inplace=True)
irvine_total.drop(axis=1, columns=['index'], inplace=True)
irvine_total.head(20)

,Type,Total
0,Park,173
1,Bakery,106
2,Coffee Shop,105
3,Sandwich Place,93
4,Chinese Restaurant,88
5,Korean Restaurant,87
6,Pizza Place,76
7,Fast Food Restaurant,76
8,Bank,69
9,Shopping Mall,68


One of the must look for the most frequent kind of venue, which means it has a large demand. 

In [22]:
irvine_total[irvine_total['Type'].str.contains('Restaurant')].head(10)

,Type,Total
4,Chinese Restaurant,88
5,Korean Restaurant,87
7,Fast Food Restaurant,76
10,Japanese Restaurant,58
12,Mexican Restaurant,55
24,American Restaurant,31
31,Indian Restaurant,24
34,Seafood Restaurant,22
36,Vegetarian / Vegan Restaurant,21
40,Asian Restaurant,19


But where should one open this type of business? The best guess would be around a village that has no other venues of this type.

In [23]:
type_of_venue = 'Chinese'
irvine_without_venue = irvine_grouped[irvine_grouped[type_of_venue + ' Restaurant'] == 0]['Neighborhood']
irvine_without_venue.reset_index()
irvine_without_venue

1             Columbus Grove
11             Lambert Ranch
12                 Northpark
18             Orchard Hills
23           Portola Springs
24                Quail Hill
28              Shady Canyon
30                The Colony
32               The Willows
33              Turtle Ridge
34               Turtle Rock
35          University Hills
36           University Park
37    University Town Center
42                Woodbridge
Name: Neighborhood, dtype: object

The big concern is to only open a restaurant where there is already a great concentration of venues. So one can make a list of the villages with the largest number of restaurants around them. Those are the top 10.

In [24]:
irvine_counter = irvine_venues[irvine_venues['Venue Category'].str.contains('Restaurant')].groupby('Neighborhood').count().reset_index()
irvine_counter.drop(irvine_counter.columns.difference(['Neighborhood', 'Venue Longitude']), 1, inplace=True)
irvine_counter.columns = ['Neighborhood', 'Total']
irvine_counter.sort_values(by='Total', ascending=False, inplace=True)
irvine_counter.reset_index(inplace=True)
irvine_counter.drop(axis=1, columns=['index'], inplace=True)
irvine_counter.head(10)

,Neighborhood,Total
0,El Camino Real,33
1,The Willows,30
2,Northwood,28
3,College Park,26
4,Harvard Square,26
5,Northpark Square,25
6,Westpark,24
7,Racquet Club,24
8,Walnut,24
9,Cypress Village,23


Crossing the top 10 with the villages where there is no Chinese restaurant, it leads to this final location.

In [25]:
final_location = ''
for i in range(0, irvine_counter.shape[0]):  
  if len(irvine_without_venue[irvine_without_venue == irvine_counter.at[i, 'Neighborhood']]) != 0:        
    print('{} has {} venues but no {} Restaurant.'.format(irvine_counter.at[i, 'Neighborhood'], irvine_counter.at[i, 'Total'], type_of_venue))
    final_location = irvine_counter.at[i, 'Neighborhood']
    break

The Willows has 30 venues but no Chinese Restaurant.


# Results

The above results were limited to the maximum number of venues per village. This limitation is due to the free Foursquare account. A more accurate result would be reached if a larger limit was allowed.

# Discussion

The assumption that Chinese restaurants would be a good choice was only based on the fact that this is the most frequent kind of restaurant present within the city, leading to the idea that there is big demand for this kind of food experience.

# Conclusion

Based on the available API, the free account limitation and the given time, one can say that, for a immigrant to open a restaurant in Irvine, California, it would be a good choice to open it around The Willows, where even though it is well served with several restaurants, none of them is a Chinese restaurant, which is the most frequent kind of restaurant in the city.

In [26]:
irvine_venues[(irvine_venues['Neighborhood'] == final_location) & (irvine_venues['Venue Category'].str.contains('Restaurant'))]
final_location_restaurants = irvine_venues[(irvine_venues['Neighborhood'] == final_location) & (irvine_venues['Venue Category'].str.contains('Restaurant'))]
final_location_restaurants.reset_index(inplace=True)
final_location_restaurants.drop('index', 1, inplace=True)

final_location_restaurants_coord = final_location_restaurants[['Venue Latitude', 'Venue Longitude']]
final_location_restaurants_coord.reset_index(inplace=True)
final_location_restaurants_coord.drop('index', 1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
final_location_restaurants


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Willows,33.685697,-117.825982,House of Kabob II,33.692771,-117.826969,Middle Eastern Restaurant
1,The Willows,33.685697,-117.825982,Tokyo Table,33.687735,-117.833669,Japanese Restaurant
2,The Willows,33.685697,-117.825982,Pepper Lunch,33.688403,-117.832805,Japanese Restaurant
3,The Willows,33.685697,-117.825982,Chick-fil-A,33.695005,-117.826469,Fast Food Restaurant
4,The Willows,33.685697,-117.825982,CoCo Ichibanya,33.687859,-117.833920,Japanese Restaurant
5,The Willows,33.685697,-117.825982,Arby's,33.692591,-117.826182,Fast Food Restaurant
6,The Willows,33.685697,-117.825982,Pho Saigon Pearl,33.688337,-117.833245,Asian Restaurant
7,The Willows,33.685697,-117.825982,Pei Wei,33.695550,-117.827770,Asian Restaurant
8,The Willows,33.685697,-117.825982,Cava Grill,33.685122,-117.810671,Mediterranean Restaurant
9,The Willows,33.685697,-117.825982,The Kickin' Crab,33.688517,-117.832769,Cajun / Creole Restaurant


In [28]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(final_location_restaurants_coord)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 2, 2, 3, 2, 3, 2, 3, 1, 2, 1, 1, 0, 2, 2, 3, 3, 1, 2, 3, 1, 1,
       2, 1, 2, 1, 4, 3, 1, 0], dtype=int32)

In [29]:
# add clustering labels
final_location_restaurants.insert(0, 'Cluster Labels', kmeans.labels_)
final_location_restaurants.reset_index(inplace=True)
final_location_restaurants.drop('index', 1, inplace=True)
final_location_restaurants.sort_values(by='Cluster Labels', inplace=True)
final_location_restaurants.reset_index(inplace=True)
final_location_restaurants.drop('index', 1, inplace=True)
final_location_restaurants

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,The Willows,33.685697,-117.825982,Del Taco,33.678408,-117.834592,Fast Food Restaurant
1,0,The Willows,33.685697,-117.825982,Rubio's Coastal Grill,33.676246,-117.832989,Mexican Restaurant
2,1,The Willows,33.685697,-117.825982,Paradise Bowls,33.684819,-117.810564,New American Restaurant
3,1,The Willows,33.685697,-117.825982,Sagami Japanese Restaurant,33.684284,-117.812013,Japanese Restaurant
4,1,The Willows,33.685697,-117.825982,Chipotle Mexican Grill,33.685467,-117.811200,Mexican Restaurant
5,1,The Willows,33.685697,-117.825982,Olive Garden,33.682510,-117.813955,Italian Restaurant
6,1,The Willows,33.685697,-117.825982,Cava Grill,33.685122,-117.810671,Mediterranean Restaurant
7,1,The Willows,33.685697,-117.825982,Cafe Rio,33.683134,-117.812997,Mexican Restaurant
8,1,The Willows,33.685697,-117.825982,Urban Plates,33.684752,-117.810786,New American Restaurant
9,1,The Willows,33.685697,-117.825982,California Fish Grill,33.684252,-117.809244,Seafood Restaurant


In [30]:
# create map
map_clusters = folium.Map(location=[final_location_restaurants.at[0, 'Neighborhood Latitude'], final_location_restaurants.at[0, 'Neighborhood Longitude']], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.Reds(np.linspace(0.5, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_location_restaurants['Venue Latitude'], final_location_restaurants['Venue Longitude'], final_location_restaurants['Neighborhood'], final_location_restaurants['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)

    folium.CircleMarker(
        [lat, lon],
        radius=10, #(cluster+1),
        popup=label,
        color=rainbow[(cluster)],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
final_location_restaurants[['Cluster Labels', 'Venue Category']].sort_values(by='Cluster Labels')

,Cluster Labels,Venue Category
0,0,Fast Food Restaurant
1,0,Mexican Restaurant
2,1,New American Restaurant
3,1,Japanese Restaurant
4,1,Mexican Restaurant
5,1,Italian Restaurant
6,1,Mediterranean Restaurant
7,1,Mexican Restaurant
8,1,New American Restaurant
9,1,Seafood Restaurant


In [32]:
rainbow

['#fb694a', '#ee3a2c', '#ca181d', '#a30f15', '#67000d']